#### Import statements

In [ ]:
from PIL import Image
import struct
import os
import random
import numpy as np
import time

#### Connect to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#### Height and with of frames

In [ ]:
width = 352
height = 288

#### Create directories if not already existing.

In [ ]:
os.makedirs("drive/MyDrive/CSCI576", exist_ok=True)
os.makedirs("drive/MyDrive/CSCI576/Project", exist_ok=True)
os.makedirs("drive/MyDrive/CSCI576/Project/QueryFiles", exist_ok=True)

#### Add paths for original video rgb files folder, preprocessed rgb files folder and the output folder where query videos are generated.

In [ ]:
input_folder = "drive/MyDrive/576_final_PA/Videos/RGB_Files/"
preprocess_folder = "drive/MyDrive/CSCI576/Project/Preprosessing/"
output_folder = "drive/MyDrive/CSCI576/Project/QueryFiles/"

#### Helper functions to generate query test videos

In [ ]:
# Append bytes to query rgb file
def append_to_rgb_file(output_file_path, data):
    with open(output_file_path, "ab") as file:
        file.write(bytes(data))

# Get number of frames in the video -> which is equivalent to the length of pre-processed RGB files
def get_file_length(file_name):
    count = 0
    with open(file_name, 'r') as file:
        for line in file:
          count+=1
    return count

# Generate query video given the input video rgb, start and end frame numbers to be included in query and the path where query video should be saved
def generate_query_video(file_path, start_frame_number, end_frame_number, output_file_path):
    with open(file_path, 'rb') as file:
      current_frame_count = 0

      # Skip frames in batches of 500 to get to start frame
      while current_frame_count+500 < start_frame_number:
        file.read(width*height*3*500)
        current_frame_count += 500

      # Skip frames in batches of 1 to get to start frame
      while current_frame_count < start_frame_number:
        file.read(width*height*3)
        current_frame_count += 1

      image_array = []
      pixel_count = 0
      try:
        while True:
            # Unpack R,G,B bytes and convert them int
            r = struct.unpack('B', file.read(1))[0]
            g = struct.unpack('B', file.read(1))[0]
            b = struct.unpack('B', file.read(1))[0]

            image_array.append(r)
            image_array.append(g)
            image_array.append(b)

            pixel_count += 1

            # If a frame is constructed, increament frame count
            if pixel_count % (width * height) == 0:
                current_frame_count += 1
                pixel_count = 0
                if (current_frame_count > end_frame_number):
                    break
      except Exception as e:
        # Handle expection as if the read reaches last byte EOF is only available to be read
        print(f"An unexpected error occurred: {e}")
      append_to_rgb_file(output_file_path, image_array)

# Add frame lengths to a dictionary
frame_length = {}

for filename in os.listdir(preprocess_folder):
    input_file_path = os.path.join(preprocess_folder, filename)
    frame_length[filename[:-4]] = get_file_length(input_file_path)

#### Code pieces to generate query videos

In [ ]:
# Generate query videos starting at first frame

max_frames = 600
start_frame_number = 0

for filename in os.listdir(input_folder):
    print("****************************************************************************************")
    print("Processing: ", filename)
    input_file_path = os.path.join(input_folder, filename)
    # Assign start frame number as 0
    random_frame_number = start_frame_number
    output_file_name = filename[:-4] + "_" + str(random_frame_number) + "_" + str(max_frames) + ".rgb"
    print("Output file: ", output_file_name)
    output_file_path = os.path.join(output_folder, output_file_name)
    if (os.path.exists(output_file_path)):
        os.remove(output_file_path)
    generate_query_video(input_file_path, random_frame_number, random_frame_number + max_frames, output_file_path)

In [ ]:
# Generate query videos consisting of last 20 seconds of original video

max_frames = 600

for filename in os.listdir(input_folder):
    print("****************************************************************************************")
    print("Processing: ", filename)
    input_file_path = os.path.join(input_folder, filename)
    random_frame_number = frame_length[filename[:-4]] - max_frames
    output_file_name = filename[:-4] + "_" + str(random_frame_number) + "_" + str(max_frames) + ".rgb"
    print("Output file: ", output_file_name)
    output_file_path = os.path.join(output_folder, output_file_name)
    if (os.path.exists(output_file_path)):
        os.remove(output_file_path)
    generate_query_video(input_file_path, random_frame_number, random_frame_number + max_frames, output_file_path)

In [ ]:
# Generate query videos frome a random frame

max_frames = 600

for filename in os.listdir(input_folder):
    print("****************************************************************************************")
    print("Processing: ", filename)
    input_file_path = os.path.join(input_folder, filename)
    random_frame_number = random.randint(1, frame_length[filename[:-4]] - max_frames)
    output_file_name = filename[:-4] + "_" + str(random_frame_number) + "_" + str(max_frames) + ".rgb"
    print("Output file: ", output_file_name)
    output_file_path = os.path.join(output_folder, output_file_name)
    if (os.path.exists(output_file_path)):
        os.remove(output_file_path)
    generate_query_video(input_file_path, random_frame_number, random_frame_number + max_frames, output_file_path)